In [1]:
#imports for data processing and also for model making and running
import pandas as pd
import numpy as np


import cv2
from imutils import url_to_image
from urllib.error import HTTPError

from keras.utils import get_file
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.applications import VGG19 as v
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers

from sklearn.preprocessing import LabelEncoder

In [2]:
#imports for loop updates
from IPython.display import clear_output
import timeit

In [3]:
from google.colab import files
uploaded = files.upload()

Saving image-Sentiment-polarity-DFE.csv to image-Sentiment-polarity-DFE.csv


In [4]:
df = pd.read_csv('./image-Sentiment-polarity-DFE.csv')

In [5]:
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,which_of_these_sentiment_scores_does_the_above_image_fit_into_best,which_of_these_sentiment_scores_does_the_above_image_fit_into_best:confidence,imageurl,which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold,which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold_reason
0,694550568,True,golden,47,NaN,Positive,0.6395,http://farm1.staticflickr.com/20/71235476_3103...,Highly positive\nPositive,"He is smiling. => happy, joy"
1,694550569,True,golden,36,NaN,Positive,0.6458,http://farm4.staticflickr.com/3096/2584485524_...,Highly positive\nPositive,A beautiful scenery. => serenity
2,694550570,True,golden,35,NaN,Negative,0.8420,http://farm3.staticflickr.com/2880/11733997835...,Negative\nHighly negative,"They look serious. => sadness, fear"
3,694550571,True,golden,41,NaN,Positive,0.7216,http://farm4.staticflickr.com/3022/2286937150_...,Highly positive\nPositive,"A cute puppy. => happy, joy"
4,694550572,True,golden,46,NaN,Positive,0.8801,http://farm1.staticflickr.com/163/357381033_d0...,Highly positive\nPositive,"She looks fun. => fun, joy, happy"
...,...,...,...,...,...,...,...,...,...,...
15608,694566363,False,finalized,3,3/26/15 1:59,Positive,0.6705,http://farm3.static.flickr.com/2198/1572103607...,NaN,NaN
15609,694566364,False,finalized,3,3/26/15 2:13,Positive,0.3373,http://farm1.static.flickr.com/100/263134442_6...,NaN,NaN
15610,694566365,False,finalized,3,3/26/15 2:54,Positive,0.6663,http://farm2.static.flickr.com/1360/1392747419...,NaN,NaN
15611,694566366,False,finalized,3,3/26/15 2:46,Negative,0.6760,http://farm3.static.flickr.com/2077/1925076121...,NaN,NaN


In [6]:
data = df[['which_of_these_sentiment_scores_does_the_above_image_fit_into_best','imageurl']]

In [7]:
data.rename(columns = {'which_of_these_sentiment_scores_does_the_above_image_fit_into_best':'sentiment'}, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
data

,sentiment,imageurl
0,Positive,http://farm1.staticflickr.com/20/71235476_3103...
1,Positive,http://farm4.staticflickr.com/3096/2584485524_...
2,Negative,http://farm3.staticflickr.com/2880/11733997835...
3,Positive,http://farm4.staticflickr.com/3022/2286937150_...
4,Positive,http://farm1.staticflickr.com/163/357381033_d0...
...,...,...
15608,Positive,http://farm3.static.flickr.com/2198/1572103607...
15609,Positive,http://farm1.static.flickr.com/100/263134442_6...
15610,Positive,http://farm2.static.flickr.com/1360/1392747419...
15611,Negative,http://farm3.static.flickr.com/2077/1925076121...


In [9]:
data.sort_values(by='sentiment',ascending=False,inplace=True,ignore_index=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [10]:
data

,sentiment,imageurl
0,Positive,http://farm1.staticflickr.com/20/71235476_3103...
1,Positive,http://farm4.static.flickr.com/3164/2595593469...
2,Positive,http://farm4.static.flickr.com/3279/2676243193...
3,Positive,http://farm1.static.flickr.com/125/355684874_e...
4,Positive,http://farm1.static.flickr.com/64/176433529_cb...
...,...,...
15608,Highly negative,http://farm4.static.flickr.com/3037/2599423131...
15609,Highly negative,http://farm3.static.flickr.com/2259/2284903984...
15610,Highly negative,http://farm7.staticflickr.com/6211/6295863341_...
15611,Highly negative,http://farm5.staticflickr.com/4027/4375152825_...


In [11]:
x = data.iloc[:1000,:]

In [12]:
y = data.iloc[13000:14000,:]

In [13]:
truncated_data = x

In [14]:
truncated_data = truncated_data.append(y,ignore_index=True)

In [15]:
truncated_data

,sentiment,imageurl
0,Positive,http://farm1.staticflickr.com/20/71235476_3103...
1,Positive,http://farm4.static.flickr.com/3164/2595593469...
2,Positive,http://farm4.static.flickr.com/3279/2676243193...
3,Positive,http://farm1.static.flickr.com/125/355684874_e...
4,Positive,http://farm1.static.flickr.com/64/176433529_cb...
...,...,...
1995,Negative,http://farm1.static.flickr.com/151/423363936_1...
1996,Negative,http://farm3.static.flickr.com/2197/2452771603...
1997,Negative,http://farm1.static.flickr.com/63/199219063_09...
1998,Negative,http://farm3.static.flickr.com/2396/2172284070...


In [21]:
truncated_data = truncated_data.sample(frac=1,ignore_index=True)

In [22]:
truncated_data

,sentiment,imageurl
0,Negative,http://farm3.static.flickr.com/2003/2320950322...
1,Positive,http://farm3.static.flickr.com/2019/2126157105...
2,Positive,http://farm2.static.flickr.com/1208/605667281_...
3,Positive,http://farm1.static.flickr.com/197/494669271_b...
4,Negative,http://farm1.static.flickr.com/159/388599992_9...
...,...,...
1995,Positive,http://farm1.static.flickr.com/99/310737223_68...
1996,Positive,http://farm3.static.flickr.com/2279/2246403516...
1997,Negative,http://farm3.static.flickr.com/2267/2097660536...
1998,Negative,http://farm3.static.flickr.com/2196/2102009201...


In [23]:
truncated_data['sentiment'].value_counts()

Negative    1000
Positive    1000
Name: sentiment, dtype: int64

In [24]:
training_images = truncated_data.iloc[:1000,:]
test_images = truncated_data.iloc[1000:,:]

print("Shape of new dataframes - {} , {}".format(training_images.shape, test_images.shape))

Shape of new dataframes - (1000, 2) , (1000, 2)


In [25]:
training_images['sentiment'].value_counts()

Positive    507
Negative    493
Name: sentiment, dtype: int64

In [26]:
test_images['sentiment'].value_counts()

Negative    507
Positive    493
Name: sentiment, dtype: int64

In [27]:
start = timeit.default_timer()

def get_data(d):
  x = []
  for i in range(len(d)):
    clear_output(wait=True)
    try:
      img = url_to_image(d[i])
    except HTTPError as err:
      if err.code == 404 or err.code == 410 or err.code == 104:
        training_images.drop([i], inplace=True)
        continue
      else:
       raise
    image = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
    x.append(image)

    stop = timeit.default_timer()

    if(i/len(d)*100)<5:
      expected_time = "Calculating..."
    else:
      time_perc =  timeit.default_timer()
      expected_time = np.round(((time_perc-start)/(i/len(d)))/60,2)

      print("Current Progress: ",np.round(i/len(training_images) * 100,2),"%")
      print("Current Run Time: ",np.round((stop-start) / 60,2),"minutes")
      print("Expected Run Time: ",expected_time,"minutes")
  x = np.array(x)
  return x

In [28]:
training_data = get_data(training_images['imageurl'])

Current Progress:  110.63 %
Current Run Time:  3.13 minutes
Expected Run Time:  3.13 minutes


In [29]:
from sklearn.preprocessing import LabelEncoder
def encode(t):
  encoder = LabelEncoder()
  
  encoder_df = pd.DataFrame(encoder.fit_transform(t[['sentiment']]))

  t = t.join(encoder_df)

  return t

In [30]:
training_images.reset_index(inplace=True)

In [31]:
training_images = encode(training_images)

training_images = training_images.dropna()

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
training_images

,index,sentiment,imageurl,0
0,0,Negative,http://farm3.static.flickr.com/2003/2320950322...,0
1,1,Positive,http://farm3.static.flickr.com/2019/2126157105...,1
2,2,Positive,http://farm2.static.flickr.com/1208/605667281_...,1
3,3,Positive,http://farm1.static.flickr.com/197/494669271_b...,1
4,6,Positive,http://farm1.static.flickr.com/148/386488418_2...,1
...,...,...,...,...
898,995,Negative,http://farm4.static.flickr.com/3226/2350113833...,0
899,996,Negative,http://farm1.static.flickr.com/121/314036511_d...,0
900,997,Negative,http://farm1.static.flickr.com/95/233313319_ad...,0
901,998,Negative,http://farm4.static.flickr.com/3085/2412068258...,0


In [33]:
training_images.drop('index',axis=1,inplace=True)

In [34]:
training_images

,sentiment,imageurl,0
0,Negative,http://farm3.static.flickr.com/2003/2320950322...,0
1,Positive,http://farm3.static.flickr.com/2019/2126157105...,1
2,Positive,http://farm2.static.flickr.com/1208/605667281_...,1
3,Positive,http://farm1.static.flickr.com/197/494669271_b...,1
4,Positive,http://farm1.static.flickr.com/148/386488418_2...,1
...,...,...,...
898,Negative,http://farm4.static.flickr.com/3226/2350113833...,0
899,Negative,http://farm1.static.flickr.com/121/314036511_d...,0
900,Negative,http://farm1.static.flickr.com/95/233313319_ad...,0
901,Negative,http://farm4.static.flickr.com/3085/2412068258...,0


In [35]:
training_label = training_images.iloc[:,2:3]

In [36]:
training_label

,0
0,0
1,1
2,1
3,1
4,1
...,...
898,0
899,0
900,0
901,0


In [38]:
training_label_categorical = tf.keras.utils.to_categorical(training_label,num_classes=2)
training_label_categorical

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [39]:
training_label_categorical.shape

(903, 2)

In [40]:
training_data.shape

(903, 224, 224, 3)

In [41]:
base_model_1 = v(input_shape=(224,224,3), weights='imagenet', include_top=False)
base_model_1.trainable = False #freeze the layers
base_model_1.summary()

80134624/80134624 [==============================] - 3s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [42]:
model = tf.keras.Sequential([
    base_model_1,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(1e-4),
    activity_regularizer=regularizers.L2(1e-5)),
    Dense(2, activation='softmax')
])
model.summary()

model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), metrics = ['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 20,288,066
Trainable params: 263,682
Non-trainable params: 20,024,384
______________________________________

In [43]:
training_data = training_data*255

In [44]:
training_data

array([[[[175, 169, 172],
         [175, 170, 172],
         [174, 169, 171],
         ...,
         [180, 175, 176],
         [182, 179, 179],
         [179, 176, 177]],

        [[174, 173, 173],
         [175, 174, 174],
         [174, 173, 173],
         ...,
         [197, 192, 193],
         [186, 182, 181],
         [190, 185, 185]],

        [[174, 174, 174],
         [175, 175, 175],
         [174, 174, 173],
         ...,
         [190, 188, 188],
         [193, 189, 189],
         [187, 183, 184]],

        ...,

        [[223, 222, 193],
         [200, 199, 165],
         [215, 214, 154],
         ...,
         [239, 239, 239],
         [234, 234, 234],
         [234, 232, 233]],

        [[212, 211, 192],
         [194, 191, 170],
         [203, 200, 167],
         ...,
         [234, 233, 233],
         [230, 229, 229],
         [226, 223, 223]],

        [[211, 210, 196],
         [187, 185, 172],
         [190, 188, 174],
         ...,
         [233, 229, 230],
        

In [45]:
history = model.fit(training_data,training_label_categorical, batch_size=5, epochs=64, validation_split=0.25)

Epoch 1/64
136/136 [==============================] - 19s 72ms/step - loss: 2.7239 - acc: 0.5510 - val_loss: 1.2504 - val_acc: 0.6593
Epoch 2/64
136/136 [==============================] - 8s 62ms/step - loss: 1.2674 - acc: 0.6765 - val_loss: 0.9376 - val_acc: 0.5929
Epoch 3/64
136/136 [==============================] - 8s 62ms/step - loss: 0.8975 - acc: 0.6898 - val_loss: 0.8643 - val_acc: 0.5752
Epoch 4/64
136/136 [==============================] - 9s 63ms/step - loss: 0.7551 - acc: 0.7548 - val_loss: 0.8025 - val_acc: 0.6726
Epoch 5/64
136/136 [==============================] - 9s 64ms/step - loss: 0.7047 - acc: 0.7415 - val_loss: 0.8225 - val_acc: 0.6726
Epoch 6/64
136/136 [==============================] - 9s 64ms/step - loss: 0.6510 - acc: 0.7637 - val_loss: 0.7765 - val_acc: 0.6858
Epoch 7/64
136/136 [==============================] - 9s 65ms/step - loss: 0.6708 - acc: 0.7563 - val_loss: 0.8823 - val_acc: 0.6504
Epoch 8/64
136/136 [==============================] - 9s 66ms/step -

In [46]:
 training_images.shape

(903, 3)

In [47]:
model.evaluate(training_data,training_label_categorical)

29/29 [==============================] - 9s 226ms/step - loss: 0.4310 - acc: 0.9136


[0.4309696555137634, 0.9136212468147278]

In [48]:
test_images = truncated_data.iloc[1000:,:]
test_images.reset_index(inplace=True)
test_images

,index,sentiment,imageurl
0,1000,Positive,http://farm1.static.flickr.com/26/62190334_ade...
1,1001,Negative,http://farm2.static.flickr.com/1092/1401073618...
2,1002,Positive,http://farm1.static.flickr.com/147/396718601_a...
3,1003,Positive,http://farm2.static.flickr.com/1067/710771991_...
4,1004,Negative,http://farm4.static.flickr.com/3293/2397993330...
...,...,...,...
995,1995,Positive,http://farm1.static.flickr.com/99/310737223_68...
996,1996,Positive,http://farm3.static.flickr.com/2279/2246403516...
997,1997,Negative,http://farm3.static.flickr.com/2267/2097660536...
998,1998,Negative,http://farm3.static.flickr.com/2196/2102009201...


In [49]:
test_images.drop('index',axis=1,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [50]:
test_images

,sentiment,imageurl
0,Positive,http://farm1.static.flickr.com/26/62190334_ade...
1,Negative,http://farm2.static.flickr.com/1092/1401073618...
2,Positive,http://farm1.static.flickr.com/147/396718601_a...
3,Positive,http://farm2.static.flickr.com/1067/710771991_...
4,Negative,http://farm4.static.flickr.com/3293/2397993330...
...,...,...
995,Positive,http://farm1.static.flickr.com/99/310737223_68...
996,Positive,http://farm3.static.flickr.com/2279/2246403516...
997,Negative,http://farm3.static.flickr.com/2267/2097660536...
998,Negative,http://farm3.static.flickr.com/2196/2102009201...


In [51]:
start = timeit.default_timer()


test_data = []
d = test_images['imageurl']
for i in range(len(d)):
  clear_output(wait=True)

  try:
    img = url_to_image(d[i])
  except HTTPError  as err:
    if err.code == 404 or err.code == 410 or err.code == 104:
      d.drop([i], inplace=True)
    else:
      raise
  image = cv2.resize(img,(224,224) ,interpolation = cv2.INTER_AREA)
  test_data.append(image)

  stop = timeit.default_timer()

  if(i/len(test_images)*100)<5:
    expected_time = "Calculating..."
  else:
    time_perc =  timeit.default_timer()
    expected_time = np.round(((time_perc-start)/(i/len(test_images)))/60,2)

    print("Current Progress: ",np.round(i/len(test_images) * 100,2),"%")
    print("Current Run Time: ",np.round((stop-start) / 60,2),"minutes")
    print("Expected Run Time: ",expected_time,"minutes")
test_data = np.array(test_data)

Current Progress:  99.9 %
Current Run Time:  3.0 minutes
Expected Run Time:  3.01 minutes


In [52]:
print("Shape of new dataframes - {} , {}".format(test_data.shape, test_images.shape))

Shape of new dataframes - (1000, 224, 224, 3) , (1000, 2)


In [53]:
test_data=test_data*255

In [54]:
test_images

,sentiment,imageurl
0,Positive,http://farm1.static.flickr.com/26/62190334_ade...
1,Negative,http://farm2.static.flickr.com/1092/1401073618...
2,Positive,http://farm1.static.flickr.com/147/396718601_a...
3,Positive,http://farm2.static.flickr.com/1067/710771991_...
4,Negative,http://farm4.static.flickr.com/3293/2397993330...
...,...,...
995,Positive,http://farm1.static.flickr.com/99/310737223_68...
996,Positive,http://farm3.static.flickr.com/2279/2246403516...
997,Negative,http://farm3.static.flickr.com/2267/2097660536...
998,Negative,http://farm3.static.flickr.com/2196/2102009201...


In [55]:
encoder = LabelEncoder()
encoder_df = pd.DataFrame(encoder.fit_transform(test_images[['sentiment']]))
test_images = test_images.join(encoder_df)
test_images.drop('sentiment',axis = 1,inplace=True)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
test_label = test_images.iloc[:,1:]

In [57]:
test_label

,0
0,1
1,0
2,1
3,1
4,0
...,...
995,1
996,1
997,0
998,0


In [61]:
test_label_categorical = tf.keras.utils.to_categorical(test_label,num_classes=2)
test_label_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [62]:
hist = model.fit(test_data,test_label_categorical, batch_size=5, epochs=64, validation_split=0.25)

Epoch 1/64
150/150 [==============================] - 9s 63ms/step - loss: 1.2726 - acc: 0.6213 - val_loss: 0.8879 - val_acc: 0.5760
Epoch 2/64
150/150 [==============================] - 9s 63ms/step - loss: 0.8074 - acc: 0.6800 - val_loss: 0.8474 - val_acc: 0.6480
Epoch 3/64
150/150 [==============================] - 10s 66ms/step - loss: 0.7693 - acc: 0.6813 - val_loss: 0.8219 - val_acc: 0.6680
Epoch 4/64
150/150 [==============================] - 10s 64ms/step - loss: 0.7297 - acc: 0.6973 - val_loss: 0.8385 - val_acc: 0.6320
Epoch 5/64
150/150 [==============================] - 10s 66ms/step - loss: 0.6733 - acc: 0.7560 - val_loss: 0.8280 - val_acc: 0.6600
Epoch 6/64
150/150 [==============================] - 10s 65ms/step - loss: 0.6831 - acc: 0.7533 - val_loss: 0.8427 - val_acc: 0.6560
Epoch 7/64
150/150 [==============================] - 10s 65ms/step - loss: 0.6801 - acc: 0.7680 - val_loss: 0.7865 - val_acc: 0.6680
Epoch 8/64
150/150 [==============================] - 10s 64ms/s

In [63]:
model.evaluate(training_data,training_label_categorical)

29/29 [==============================] - 5s 186ms/step - loss: 0.8398 - acc: 0.7043


[0.839820146560669, 0.7043189406394958]

In [64]:
model.evaluate(test_data,test_label_categorical)

32/32 [==============================] - 7s 216ms/step - loss: 0.4497 - acc: 0.8800


[0.4496575891971588, 0.8799999952316284]

In [65]:
def img_data(d): 
  tx = []
  img = url_to_image(d)
  image = cv2.resize(img,(224,224) ,interpolation = cv2.INTER_AREA)
  tx.append(image)
  tx = np.array(tx)
  return tx

In [66]:
def sentiment(p):
  if(p == 0):
    return 'Negative'
  else:
    return "Positive"

In [67]:
def get_prediction(x):
  tx = img_data(x)
  res = np.argmax(model.predict(tx))
  return sentiment(res)

In [68]:
l = 'https://th.bing.com/th/id/OIP.b-YLmeGfdPFHAAO-NyD2owHaHa?w=156&h=180&c=7&r=0&o=5&pid=1.7'
print("Predicted: ", get_prediction(l))

1/1 [==============================] - 0s 185ms/step
Predicted:  Negative


In [70]:
k = 'https://th.bing.com/th/id/OIP.AC14ntaV9uLkfI_I8Wx4PAHaEs?w=289&h=184&c=7&r=0&o=5&pid=1.7'
print("Predicted: ", get_prediction(k))

1/1 [==============================] - 0s 18ms/step
Predicted:  Negative


In [71]:
i = 'https://th.bing.com/th/id/OIP.G7i9YVdJ8A5bRqoOdPQVfAAAAA?w=116&h=180&c=7&r=0&o=5&pid=1.7'
print("predicted: ",get_prediction(i))

1/1 [==============================] - 0s 16ms/step
predicted:  Negative


In [72]:
o = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAsJCQcJCQcJCQkJCwkJCQkJCQsJCwsMCwsLDA0QDBEODQ4MEhkSJRodJR0ZHxwpKRYlNzU2GioyPi0pMBk7IRP/2wBDAQcICAsJCxULCxUsHRkdLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCz/wAARCAC0ASsDASIAAhEBAxEB/8QAGwAAAQUBAQAAAAAAAAAAAAAABAABAgMFBgf/xAA5EAABBAEDAgUCBAYBAgcAAAABAAIDESEEEjFBUQUTImFxMoEGFEKRI1KhscHRUyRyNENiY4Lh8P/EABoBAAMBAQEBAAAAAAAAAAAAAAABAgMEBQb/xAAjEQEBAAIDAQACAwADAAAAAAAAAQIRAxIhMRNBBDJRIkJh/9oADAMBAAIRAxEAPwDiRpnxuDnvFDOEQHBuQbBFHuq53kHj2CqDzQNDPThc2MdHbYl0hYwvbe12KT7tsJe/F8fCFc4uocAdLV8r90cMdg19VKtHK0dBIx0b43AfxBTWk/VaCbJLFLJA8WG3YFZ91KIQjY5hPmMugD2RDYWztkllHrrYwggFo7lA2pZJE17W7SHHhbej8MMojcHAl3qIGTRWJE0wiYTMLmhp8pzP5vv0XWfhksfFAQRueXUXc4NVSVbcUluq09B4VKwENwD3OR9kUPCpNNI6Rsm5sh9QP6b5W1BC+uQfcJyyvrIJ9llba75jjLqMyTSNcaa7gC+5+yDm0wbuGVqybGvsGsUcoF0zC4h3Hfusq2k0w5YtpwqwCOqO1rmbgWjHCBc5tKLG0EQ62bTua4Cy3IP+Cj9T4j4brtO9k0LhKQNufpd1IKwXvwaKHdM4AVyE8bplnNxZIzy3EA23lpQOrJtuVb5xc7JtVav9KiuTIG8mj8KvR3bvkqbuHfCWiAo/KP0yn0XZ3jKi4nup/r+yrPJTxY8/0gT3Urvqop1TmT6DKfKYcJ0lFnups5HwoKbOUBZZ7pWU1WnQCs904NKJToCdprymBS5QDn5TZ7pymGEAqPdNnuntJAco9zHdcgfuVVktPqO44I7fCrc47ybw3thIysI4N/5XoaRtMH1Nu7AyeisjedzsGgDVoXeG5JyaFf5U/Ndku9q7UEUSiY5XNedp9QIKM0kscc0s0rtzfqDOh9qWTGXHc+66fAVrJg7jGaruhcrfZqH6p0rpnxtjDjsaGgCunC3PCJW6OSIlwdE6iw1gE5orkIpQ4hvdufldD4fNGGMa/Jb9PbhRbqOji/s9CHiWlYxlPt8gtrWZPaz0QT9a4FxJJJJskrnodS2N5cXAEigD2RfnteOmeywu3o46Fy6vcCLzeEO6QHlwv3VDqAJsKkvsm6FKFrJngudkEAYQL38hSfIAT2QjpASVKt6Se4AE2hjICaSkcXYCp8p5OLQm05eAVdLG+SBszAS1pp/dvYqh0D/1Y6q5kr2Rvi/S4UfdFZZY7BO4d8J9Hw75TPwHf0S0fX5Kn9OWfRf61E8n7pyfWPmkx5PyjH4x5iTpklTnTFUnTDhOgzqTVBTakFgTJJIB0kydAKk5HCYJ0AyfsmPRK0A6SYJ0BxYaHON+lQOxt04kVnHVMX0SRf8AmlU7IeXXk4HU+69FmcEekk839lY7ZTRmuc8IdpaG7ibN0AnLmlwsn4AJT0BJeS05of3UWOZ2yVAU4gk+gJ7YCTdG/TSSl8cxaSK+61tLqHUHB5BA4Kwv4bTe8/dERSZADvhTlNtMMtV1DJhdvfZIsVwESzVk4C5qHUva4D6jxR6o+PVPBoMA72VjY7cORt/mZOhKcTOPIyssaiY87fsP8qZldi3FZWOiZjJZaO3HGfZDlwVJf1vKrLnnhTpXYW3aeaVoLAOVmeV4hIR5TCb+APuStAxSQQg6qm4vc1wKNHMp+zS6iMCjSElmG3cDhZ2p1Rc9zWn0i82OFXJqf+ncAeWmiqxwY58svwTHI6R0nUN5I4tE6Tg/Kz/DnEwycVZvuTXVaGjGHfKXJNObG+7En6h8pj1+VIj1j5THkqMWXMZOmSTYLBwkkBgJIBZU2KCm1BpprykcpcJA6dR5ToB7TqKkEAx6JJ03VAPSSe01DugOCcav24VeTjkf7VkzHsfIwtNiv2KQA9NiqXpT4yQqgRV/CkC0ABoN1TrScSDQF3/+KrD6+5xWSUHFxAaBZz2TOLQG2KdWeirImBJcx4xY3Nc3+4VjWF4FnPW+UHfEWi3USBdCzwPdXbXNI2neOlcqrytt10ycq2G/Tu+kepnycZSsOC4Rte0tcXOA5rAsZRLfNJoWReCowMbRcBlFw73OsYIPUYWeTowSjJFCyiG5+yQDm3bWn5CXnVTfKZ70T/pY6dXxfFA6U3kDizwjRDp4W7nEGgL+Vnfm+GtFfCjIZ5hRcQ2+P9JXFc5NQbqtfBFFtYQLySBnC5bX6/VTFwa5zmkeqr4CPl0kDCS6R7ic042hJPIjaXHAohoPLvstMZIw5MssmZCJZpGMDrLzkWukh8O0z4zFNVPAG68scOCFm+G6MGZs7mEjLgM0PldFFGJJAAAAOQlyZa+Hw8e56z2aSTSMkic1gDSdjmfS9p4Kt0fX7rYGh89jmFwBF7D2WbFE6F8kbxTmOLSsMrtplx3FM/WFF3P3U/1hQdyflTHJzfSSSSTYLGlOogJ0AinHITFSag0wnTcpWEAk4TWkgHT9UyQ5QEkspBOawkDZSynTWg3La/Tbx5zeWNO4DqAsYSGyQfc2unIBBB4cCD91zOuh/L6iRgy27Hwcrs4st+VlSZ5kzwxg9UhAGcC/8Lfi1vh3gwa3TRsknoF80jQ97nVnbfA7LndJN5c7XHs5ovoSKtSLXyzhpN7jg+y1ym/rXjy6+x0cv4odqWmLVaeKSI8jy23Sz5zoZRu0bnNHJYRwTyMqiXRsjZuc+v7oFjy12OP7qZJfjXPK71knI2TOf2U4HSlzW0avKK08DpGNlv02RZ4sI+PTtc00wAkH1N6p9kdN+xHSlrxTbDgdp+eyMifs3Ne2i0k2ELCWxbWPBFPxtNH5K0Rp4HReaNS1ryaqQ3nsVNVjdG80u6EhM4srII+FY2Om0aJBAwefcKxkTDy0fdZ10zdACVo9TY3kg0AMk/YKTn6xzS5rYom/++LJHw03/VHOaxoPltz/AOlAyRbT5k5LwCC2Pp/8u6Wx1oGSKeUlzpZnjP8A4eIho9rVMOm82WmiYlv1OmvHsFrHU6qZvlxsaxnANVQ9qRTooNNDQre4AknkurJR3/Q/HPoYSCEM08NGV/pHWh1JWjHoNdE1ssbw6x6iTVfFLAdKYJPzJBINi/ZTf+I3bQ3e6hwATSXW08c8cfroW6t7GVJhwxjg/dUCXznvd1BornT4w7UObGwOc5xpoa0kkrb0LJGxbpMSPdbm/wAvbhZ54WTdaXmmU1Fx+sKJ5Km4esKJGSs44ub6ZOmTpsExwkmacJ0GdOMJk/ZBLKCbqkkEjKimUlH/AO0wkE6ZJAOCnUQntIHCfCYFJMmB5oF2CVneJxtlaJWggtFE10WqyGMv/iOa1vXOSqfEZdGyCRjNuwjbY5J6LownqssZpyzhVkI3THzQ11/xIiL7lqDcLsi6JTwSPhkY9vQ+odx1C6smXHdVr6qJ8jGEXtOTSA8l5stY7aOSAcBaMWtEZt8Pm6Y5oXbb6YWnpdb4G+OUflX7SDjzHgj2WMtxd1xx5PdsxmqjGmh08YALDdHrZu1o6RxLReD1WbJp9MXgw2HX6Ggkq+F0kRp5Id1U2njL8rTdFE9wL+2TSaWBp8vZgMxj6cd7Qr9QHAer9jlRE8pb5bTTB9RJ9ZF3W7snKnOe+LDMYXgsduHW7wr49YZcv4CBkGBtcCDdgdPlRY1wwDhTfVYWxqmdo4PyobvMILqroCgml45UzIWhZ6bdhzNrLdikJqdQZZGtB+qmhDyalwaQChY5CZt/Y4tExLLP9Ohjgi2NB2kUARg2g5tB4cSXeS0E8loVun1IGxpaSKNEWRhWyzQsBLyNxsANq9vurm4VylmgcfhkMQGri2ANNdnAHFhGaaWJjXNBv1biXHr2QvnO1L2QQNJe80GCg2z1KKZpXxXG5oLwfVmxfYJZXc0zuMk2t82Pfe5v7pi9ln1N/dXjw4uAcGE2AcKQ8Me6yInED9lHVlljcgu9n8zf3TeYz+YImTw17RmMt+6i3wyQguDHkCrxwjon8dVNkiHLwpebF/MFJ2jawW5vHfCjHopdQ7bCwngn4R0HQ3mxD9YTiaH+cIp3gmpiAMsbm2L6H+yz5tKWPDa+QjpC6CDqIB/5gTfmdOP1j7WqnaUMYC5tE8J49E+Rhc1thvOCn0g6LPzWn/nH3tP+YgP6whTpXAmxVK6PRl3AJHsjpB1WHUQD9f8AdL8zB/OP6qJ0RN7WmwhZIHbiOO6OkHQb+Y0//I1L8xp/+Rv7oIaQkdSl+WOMI6QdR35jT/8AI1P+Y0//ACNWedP9vlV+RJ3CXSDqHYSXF7mueBe6gTQr2WJ4pfmWA5oLsA2MLa1G9jnNgcQHghwBr7LK8Sjk8qN73E7SAAc0CurDH3aM8716M6Ab3OZfIsfZWCF26q6qXh8LpdQSOI2Fx9+lI4sLeiu3VRjjsomOjaHNJBHZSfq9M9obI10MoN+bE0U7/uarA0+WcIDUM5wonro+TwdFrtHpvU1zpZObI/oh36yWeaSV/wCrgDhoWdkFWs7d0+sT+S3wX5x7qxsryPbqqPLcQKCenjHRTpW2hG+wEQ0tr4WfG5reTnlWOnZVArO47a45CXSBUvebwUMZxfKQkshHUXNYcglRotqsk4HsVM5aoNjmlD9n6ec0c9k4mz/FwnkiMUTXOaXG7ccNK0YdJAJNK/UTFwlcRT3bRuq+bWXDpz9cjXOcPpabJvm1pNl05LYJ4hJH6XBpF5Hyll/41xxum74bpdFJq9RDBLE1sLfPcWuBORhoI6/dNO4CaSjfqye5+VzelmfpddPqoYnxaJzzHtcCGkOxQHPuFtaomGd0U+HgMdYOC17Q5pHyEph/iMsvHX6DxLSQ6BunnhjfncOLznOETB4ppG4LIdoFAFuOb4C4tmoZtABJAxgqbtQKIbYPuo6CcjW8X8Uj1Uw8iNsbG49OLPdaPhes8ObpPy8uw7nFznnDhjhcaXyEmz9ynEr2AkuVdfC7+7bfic0Gq1Ih09CIENBrk+9J9T4lptMyHRaJgBjAGomAG6ST55oLFinc0ON24/SeoVYskn3s/KfVNzbcHizm6zS76dG1zQ5pJpwByCmkY3VeJSPaNsT5S+hwGk8LCa8ecCehR7NQ8WWvoI6jsN8U8kzMiia1rWgDBsn5WppdR4bHohpy0eYS4uJ6k8ZXLGX17ibPKkZ3nqlodtNDxHyG0YyPUc1wE+i1DYoZXYJI2ixx7rKle54ALiUt9RVZCrQ7N/w2aAt1Tpdtlj9l/wAxWW6Njp3kH0l2ENDK4Ndk590zXuDrv3yjQ7bdN4f4bDK3fLNDFE0W5z3t3O9mgppdP4c6RzIHWGmt18rBGpm4DlITytGHV1wlZVdoN1mlgjDzvBoEcDn2WTsH84/ZSlmkeTbjXyq9o7lElRbsAYxZJygPEGF8LwOgv9kU57gaUZAHNI7grqk05b6o8JhaIXGhbsuNZKukhskgdeFLw8AAsvIsf1RL2bSeVz2+uvHHeMDNZ6SCEHNFuJFLSweioe31HCNq0yTpyCcFN5QatYxAgmkDO1zTTaLR1Fq9oykikTOb6a46qt84ymkDwMg5QxPKuRlbVpnKbzSeqotJVqI7VeHm1ex9UgwTauY5Kw5aPa8kBEaRzGvIfwb4QUJ3XRUhIB8glZ2OjHLXro2wte0GKTbWQSL/AKjKqZ4WXue/89GHnaGsc0iyTm3LO0+vdFW08cgnBRjPEYCQS2jd2e6nVdM5Z+2xFo/CoYxHqJJZHk29sIBvaMNt54J5Wd4tIXyxusFxYLro0YaMquTxJmC0AkCrQTpXSvLz1x9k8Z6w5s511FsWpkj6GijmakPF0D8YWbQTtJacErTUcktjVEkb8cHsVA+o1eOiEY42DaP0jBNIB1Ga7qbiuZLAwNaP9Jmg7XFGTRhrABWVW6Oo7A6FQpnsAMh4V+O6riaN5HuivKaUEpa2yrhC4gK9kFdEWyMUBSNGypIS2lTICG4WzPBuacLInaRj3pMGia4sVjWOsq+CKowe6tEY7IAMghQ9VotzM8KTYSckIAMjuE1t7FFyR03hVeUUtBkObfKqqvup+buuxSe2kFdMc9U6dwh1Bs4cbWw5gkYHDqFhSg7t3Y3YWtpZSYQCeAuXlx1du7+PluapCI8VkIWctjNfqRRnebDWjtfVBTRFxJcTZWeNa5yT4HP8Qm3vb8JvKIB9bz/3UQrW+W0U+yP7JSObsPlmwO+CPutYxrMneYyR0KDeQ7Ncq3Ul7jkkgd0NZW0c9MklaSpmcJw4qCcIGxcDiCCERIzf62UCfqHugYpNhHZGtkjIBGCosbY3cUbJBfIVjQ7Fn91J8zQ04tDum3UAiTZW9RcbwTQRIcAFnQO9SK3ZT1pnbsU16bcbKqa5TGUEJieARaKZI5j2vYSC0gghZ+6iFc55oUUHK6Bmsj1TWNcAyXggcE9wiJmbNOSOxXMxPdggmwbC3I9eJtL5MuJWg07o4V/dRY0xyCQD+J7ly2YoC6lj6c/xgPcLqNMy2NPsFKoGMZbQREMRItWyMyPdXRNAaP6pwwmoMcTCXHNcLBcx08wDOLz+60fFZC1xaEN4ZGZJb6YTpDvy4bGz4UfLrgLTfEAzjohtnSlJhBFuNUr26faMo3TaZpJJRsum2QveR6Q27VTEOcma0GqChs9v6KT3+ZMAB1RwgNDA4CVDhy0cqogjhWAvcp7W9Vu5gtuNgjBwitA4AmInji+yi9zRgVdIWV5hMckbXOe1w44rrajPHtGvFn0ybTgAcBUzAOFcHurGSsliY9py4DHb2VcrZXtOHCurVyTz69C2WeMmeHUtJLfUO7DkfZC+u6e75uwUVPp32SyV99bcUC5kwPrBOeVtHLZV0rWPZ9YJHTlZr2bSR06LUhDKNtFoLVWH0AKORQV433TPOeBUk5TUtGOiSSo9kXFAGjfLgDNFKnJaEcCyr5OQpNlcE0z/ADHlwFDgfAVacRbq+Jl7nEkn7KVUAT1VbckeyuOa9kyWxOAwigUE27RkZtv7JKi0KwHsqgpNKRrCaLfdWAnaFRI4Ncz4VjHbgUBcx5ajWubtGeizrVzHkclI40tIQZmC8kgBdrpYz5bR7Lz6OUtcHA0QQQexGV6N4HPDr9EycEeYweVM0fpkaM/Y8hLTTGoTM2/PRKKwKpESxkPJPF4VkMV2a49ktLcj4w53nFpFDH9UX+H42udIewCF8eDRq5A04BH9lrfhaJkj3g/8YJ/2j9k05m+k1xwhmxFx46rUmgaCQeimyCJm1xFE0qsNVpdLHW54vGBws7xjVvi0z9MCKJx3oZyt0yNa0hjRYH1VhcX43ITO5oORRNHuEfCobQh0s45JXTjTuoYPAWP+HtM6acEV9LjZ9qXVeXIMWMY5UyB4zFKHMtriS4mvhVznV2GxEOu8HlYTJZo72veL5pxCJ02sdE71km+pNm/dauZfI/xNhp0TjXJbbr/ZKKbVg25kjR1tjv8AKKPimlbX1OJoYGB82iW6lrm2HA3xlAPpNaGObuadhNOsVnutoPZLG1zCNrh06eyxRT7+oHrVV+xUg6eLLZA4fykbT/RY58fb2Oji5rh5R0kMbvqYCfhAz6drQS0EfGVe3W8CUADv0/dF7oXsDmuaQVnq4umZY5/HPjdZGP7JnacTOB6gLRkgj3FwIyVKDTGyRxwEdtF02zPyPsqXaWrAGVvu0xVD4CLwiclPLijIbA2OrAJ6koXUykuMYOG8+5WlqWmON7uvAHuskREkknK2x99cnL/x8ilKirjGGAkkUoNaXkADHVaMDNFZU7TubRDR0SAObSCTUVGh2tPKKYKCDi1otSa0AhMBgKbUjVSgGRoPZWtG0YVDy18+0HLWi/7q7gfCAkE+4igmBxaRcLwg0nS7AK5XQ/hPxLUw+Jwae/4OsLopAcDcGuc13zivuuYcdzkZpZpNNJDqI/rgkZI33c03SRy+vYHQ+YPVXPRWRRuiY6rIyUPo5Y9fHp9TE7a2eCOYC+NwusI3zvLZJ6d21pNfZORu878dl3azUYzvv4FLoPwkSGPcBZc1rfgcrlPEnul1eqc4m3PJx88Lqfw29ungLzkULHe1N+lL66nZuc6wPq5PZKQxsJsXSqGrbtOaGSEG7Utfdizk2rpiZJI/Ke4gAC/bjK4DVS+bqJHdC7Hsuq1ckpheI2k2KAyLJXPN8J1fmtdLNAzeba0kkn2WHJz8eHmVaY8PJn/WOh/DjBHEXkgZAz1C2CHWfSeT0WPoJ4tExsb9hc0ODi76K7og+K+H2b1jL60WkfZGPPx/6r8PJP8Aq8K10bI9TKxgoAlDJJLpecSmySRhG1xHVJJINTTaqctFuGavARjyXMyf2SSQcQYSTRNjsVZljmhpIDgSQkklfh/tpQta6EEgE0VfC0UUklxPUx+LXAUqXtb26JJJNL8Y3iYADAPcrPDW9kkl18fx5nP/AGCan6q6VwrYABHfVJJaMFYFucT3CmALSSSC5oFK5oFJJIOLB0T8JJKTG+A+G6TxDWaoagy00gfw37enXCp10TNPqtTDHexkr2Ns2aB6lJJMKR9IVZ5KSSATOSfdEtA8v7pJJHPr0f8AC73nwnw63H0+fGP+1sjgAtnVEiGYAkXE/PXhJJN04fXneqA3v5+v/K6XwfOmA6YKSSz/AGm/Wt0dkqcETCQTZz14wkkuP+dbMfHV/FkuXoAzTP1GpjLvRHO5rWgAAC0YNNp3hu5gNEkX0JSSXzlu/r6jGSYRGTRaQ3cfODkoQ+HeHWf+nZye/wDtJJa8dumeo//Z'
print("predicted: ",get_prediction(o))

1/1 [==============================] - 0s 15ms/step
predicted:  Negative


In [73]:
p = 'https://th.bing.com/th/id/OIP.r0aOnlQ51qfje_-pilV4hAHaJI?w=131&h=180&c=7&r=0&o=5&pid=1.7'
print("predicted: ",get_prediction(p))

1/1 [==============================] - 0s 16ms/step
predicted:  Negative


In [74]:
u = 'https://th.bing.com/th/id/OIP.LLorE-TsPyNY_EAqNIDZYwHaHH?w=167&h=180&c=7&r=0&o=5&pid=1.7'
print("Predicted: ",get_prediction(u))

1/1 [==============================] - 0s 18ms/step
Predicted:  Negative


In [75]:
from PIL import Image
hun = 'https://th.bing.com/th/id/OIP.R78esYziCs7kUbBRP6Io8wAAAA?w=183&h=186&c=7&r=0&o=5&pid=1.7'
print("Predicted: ",get_prediction(hun))

1/1 [==============================] - 0s 17ms/step
Predicted:  Negative


In [80]:
model.save('best_model.h5')

In [76]:
p = "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYVFRgVFRUYGRgZFRgYGBgYGBgYGBIYGBgZGRgYGBgcIS4lHB4rIRgYJjgmKy8xNTU1GiQ7QDszPy40NTEBDAwMEA8QHhISHjQrJCw0NDQ0NDQ0NDQ0NDE0NDQ0NDQ0NDQ0NDQ0MTQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIALcBEwMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAEAAIDBQYBB//EADgQAAIBAgUDAwIFAgUEAwAAAAECAAMRBAUSITFBUWETInEGgRQyQpGhUsEVM3Kx4SNi0fCCkvH/xAAaAQADAQEBAQAAAAAAAAAAAAAAAQIDBAUG/8QAJxEAAgICAgEDBAMBAAAAAAAAAAECERIhAzFBBBNRImGRoQWB0RT/2gAMAwEAAhEDEQA/APMLTto4CPAlWc1Eemc0ya04RCwxI9MWmSCSAQsdEASOFIyYCSpFY1EFFExww5h6ASSwisMUV4wpjhhYbqEcHEWTHigNcFJVy+ErVElWuIWylFAv4CM/BywNYSP1BE5DqJCmCEkGBEIpuJLrEzfIFRBlwY7R34MdpKawnDiBFkP6SL8IO0X4cdo815BUxFo8hXEmFIdo4URAziojjLR2wyQYaInBTEAbHTgx0dMMkWQpid0CVv46c/HQ2GSLdEElRBKhMfJkx8iTkhqaLUACdBWU7Y+MONgrD3EXhcRrOJSHG+Z38bE7D3C79QRSl/EmKRbD3Ci1RapyKdtHPY7XOapyKFDscGjvUkcUKCyT1J0VZFFCgyYQK5iOIMHihQWyb1zOGsZGoiIipDtj/WM6K5kUUYrClrGTJUgIMcHkSjY7LZKkk1ypWtJFxEwlxMLDXeQF/MgatGF44wYMMDyN2g4qxjVJagxExeRu8h1REzRRCzrNEGjJ2VQjuqLVEFj1pE9IaAaHki1J0YZu0euEbtJdAIPOM0nTDd5I2HkasYHeOW8ssLgNREu6GQgx6BJsy+lp2bL/AANe0UKQ8WeexRRTVmaFFFFAoUUUUQxRTtorQCjk6BEokqpuImxpDVWEYakCd4SlIWjqNIhuJk5mijQDiaVjtIVQy5qYU34kbYU9oly6JcUVLCNll+AJjqOWkm1pammTRWKI4rNHRybe1pc4X6bVhuIs1Y1FswQBjvTY9DPRl+lk/pkyfTK9pWaH7cjzZcK5/SY8YBz+kz0+n9PgdIXTydR+mLNDXGzypMsc9DC6WSsek9TTKU/pkq5SnaGY/aZ5nS+nCekLp/SV56MmWKOknTAgRZspcSPNz9MlekhbKtJ4+Z6TWwnMrHwNzx1kuTE+P4MlRy0MeIT/AIWO01lLLgOk5Vwg6CLJh7ZkP8KBvKnG4XQCexm5q0NO9ukx2bv7mEW7IlGkCYDF6TvNBhczUi3WYtgY6nVNxK2Sm0b9cyTvFMV67d4pOTHkyiilmmT1G/TCaf07UPKmdLkhKDKS0Vpq6H0sx5BlhQ+kh1kuaKUJPwYZUk1LDk8CelYP6ZReVB+RJq/0wv5lGk+Jm+Q0XEzy84Zu0np5a7dJe5rRNN9LAeCOsO+nCpJ1cCSuYjH6qMxh8tJNjcGWJykqNVpe13T1gF+8vMMitdbC1opTbLjFMyGAwmra0Mq4RUcA2llSdErFduJWfUmJUONJ5EyQPSLD8Ih38SsxrKrWFrQzDMWQHxB6mE1fJg/gT+wDS3NrQuwXpLrAZOAB3tDKuSA3lJJBjIo8BU1Ed5ssvo7CVlHKQtpeYRbC0aWzSMWuycUp30hH3ivKLGimI4IIp0RgcCRwWPAjgIAQvtOo9xJisGddJiboBrt3jUpg7yvzitpXUDYiUtDPzoa/QyHNJ0xNpGwCCQVUgOW4/WmrxJUxoMtSTC0B402BvxaZN8u9RiR3l1nmLuNvgzuX10RLnm28L2ZSpumZ/E5AbAAfMCfImUXtNbh8wV2IFtjLdaKssqxYJ9HmDYRh+mKehtlCHpOyaF7bCUy1FF9MJoYVSLgSaliUcGx4JB+RAMqzFGd0U3sTt2PWRkdWkGlFDBbC5hAw47TGY7O2TFgMfav9xL+pniXQqwIY/wDH94lLsSkmWopCZ/Ncy9F9HcG0nxOf06blHYD23mWzXMUxD6lOy6t+4ttKtETnS0Z/PccXc/MWVY/RBMUNVz3kOHuAYkotHK5O7DKuKPqa+5mhyrMr3uZncRSChTybXMbhqjarL1jdNocZNE+a4pjWJU9YFVdiw1X+8kYWfbdibfeF4zLmXQSbsefEaqwdsv1JWjqA/TtIsqxmph4/mF4YH0Qlt9MCy3AGm4JHWOkXT0avKixJJFt/4lu67RlFBpBEe52gbpUgFa4vZvtDcP3mdx1Nncaf6pocKtlAMSQJhF4rxt5xnA572/eMZKI9RGpJFEAHgRM4E47WEgJnk/yP8j/z/RDcn+jo4eHLb6HNVMhc3iJjGM+bn6nm5HcpP8nbGEV0iGvSVxZhcSgzTJwUb0/zW2U9fAP/AJmgYwWq06OD1fNxvTv7Mjk4YSW0VH09TdqbIwIYG1jyPEL/AADpck8XMdQxQR7ng7E9vMs8SrFWHjafUek9RHmhfnyjzp8WDox2PoHSf9Vx5ltl+Whk93WRNhX31D4+0u8Bso+J1LsxUUVdPJVQsVh1FSCPiGvA672YCUx0l0FWiinYyjK5XiTTouHO4dxqPU32JmWoZu6VS6dG93m//ol9SxPrrWoKN/Ua7Dew8eYHlWQaRVatsguB+0yxVbM5ZOqKnO8X6r6wLBhKxMU6cE7HaajDfT7tRu7KqkllvzbpM/jcG6oWA9imxbvvaVFIykpdsgxeNaodTk3sB+0Jwv5Sb9N5M2Ruy0wljrGpf/2H5d9NVihdzpHVeu0HFUCUr6KRqwjywI46X/aGDJh+J9K+xsR8GX+N+m3RGbayqQBbnbrEoxSGoyfgyiMWG/EMwlPSSB1G0X+G1ERKpUlDubdPmW2Gy84hgU9thaNJIMZX0ZzDA6yexPPS01FFGYXqC326SjzLBPhqtnNyRqt3F+fvY/zN+qK9FWte6gj4Ig2vBcIN3ZUrjQqgAX6Ax2OwtWoAabAGErk61UZb6Tb2n+hujCVWAzg4dzRxIKunDfpcdCPnvFZb1p9Giy+vUp6EqHduPtLDMMUEQueBbV4HU/aZzMs7ov6NRGuwce3g78gwbH5xUqEMiKUDsrJfdrbAybdtDcoo1uGoqBrve4vfwd5FisS11CEG53+Jj8NnJTDFCxWooKhWG1RDuB4IvsY2ni/SSk4ckuoAudlN7GFk5o31WvoXUd9uByZX5ziiaKvTNyaiWt/qFxAKmdozrSQ3IXUxPHF//MpsNm/pVW1+5Hqhwq3JU78D5sYOXj5Kckeg0agI2O9pKr+2/gzMZZiiMaysGVXQsgbbe2oi3wD+0OxucqgdF2YG1zwLiGemylIsqVbWqn/tH/MjxWICKztwov8APYQVq4p00OliPaoAFyCxCrfsLkAnpIM+RmoOFF2ABAHLaSCQPNrz46d8vPlPpuv2epFYw14QDlv1Cz1hTZVAa+ki9wQCbG/OwMu3e0wf0xTapiA4B0JqLG22oggL873+02GLOpGW/KkfFxab+u4eOHMow+FZPBKTi2zJYj67tUsKYNK9tVzrI/qA4+38zStXDAMDcEXB7g7gzyOtSYOaZU6g2nT11cWE9DoE06KIdylNVNupVQDb9p6HrfR8PHGHtqm/39zl4Oacm8iXGVJovp3F+rQF/wAyEoftYr/BH7TD1cerLqVgw7jiGfSWdJSd0c2DjUtyACyEXXf9RDEj/TNvQQlCVMnlkmbp6QtBUTSYaai306hqte1xe17Xt2kTkXtcXte3W3eewZUUiZuFr/h3FmNyrdGQC4Pzz+0Nqi7A+BMF9aYxjX0sNDorKrIbh1Y3B8GxsRNZlmYo1JG1A2RQ2+4svuNutuf3hZlGVyaZaFgJ2Ur5nh39/rINQBseRsOYo7RoFfT+Trh1YDe7XueexkWeEuyYZOGOpyOijn94Pkec66LU3YM6JYMlrVLsaaabdSQP36SofOg2ISop2Wgmvb9RdXcW+EAmDdDco0i5zrD0qtLTr06NtuhEGqZMamB9NeTvf+rrJMTiaYxApmxp4hLg9ja4P3Bmgy+nopqnIUWB7gcGVYJKVmUweU1cOaLfmQW1DqnxNfVph009CJQ5vinQ4h0NgiKHLWIUqmpFRSPzFqg1E7AAbXN4b9O1HeitaoAHdFsq8KgG1v8AUSzf/IDpGONJ0ivzHJ9DrVW5ZSv7C00jqGWx4I/3keJrIilnIC7XJsBuQBufJlZj/qDDpTLiojnhVRgSzdPgeZOkO0h2Jr0MMgWs6qjE6QQST3sACZNhMPTRfUUaUK6jqBWy2vcg7jbvMziUXE4jD1KlRCi0nrVNOyU1R1sm+/JAJPNm42AGzbOneg7sCDiSadBOdFBD73PdnJA/a3Ea2ZudWQYh0xK4rEu2hS9NKRI1FQgL6QvdgFNuhYyzynN2p0xTq0nsqg3XdlXuUO5XyLyqy/CFqKNW9mFpsXZrHVVdiAERRcseF27d5u6y02TU4UKBcE+3QLd/0wVkxTe/ILluKR1103VlPW9reCOhgP1nlorUdaka03TcDWOWQHvYXA7jzM1jsSn4hvwjaAEJqu5ApkDrZh1236k7d4E2cvWVKblUp6gXdUN9t7aVvx4HNpSTFLkTTTKv1QVT2gbWJH6xfn5EObGr6aoAUcMG132cX3uOn/EEx6Wd2p6mpazpL2DW6W7/ADaCIGdlVQT7tu+/jt/4jUXZhdaLXE4tHUMQRUChe4bc+4diBbaRaWfQh/KnG/JJ/wCIOtNh7tiCdKi41M1gTtzaxG/kSMVmFx2It5vE4y8BZu8vy6mtP1tZDsmlienuCEW56yoDKEdN7u66H22ANzc8jptKjDY6ptrJCAlrXFuCwBF77tb9/MOwzl9TWVQFu7AXSnq2Flvc/wBRPgmc84zy18JGmSa6LPFYzTiEOtyFdNLsRcoCQ5t0FofmOYIGbRc6qmpB/UWa2s91B1WHg+JkcaSSWBJS+lWbYOF/MR53BsO8Zjc1UqbfmPYnYdFHi0rCT/sSnVnobZ3TNEEOC4AYL1LKwOm3kgCcP1ErGy03+Wst/sCT+4Exn09XuLMjWZgga2xYqWCL3JAYkeJaZViEw7nDvYKSTRc7BlP6G7ML2HcW+/j83ooQclTb7/PdHpcfNKSTul0axcTqF5DUqwV8SIFXxg7zzo8Lb6OtypEmIddWqwvxewvb5lXmeOChieALyDE4zuZnsxx3qPb9C8n+ojgT1vS+kcmm+kcXNypLXZO2I0oALA6ftc7n+byuqVS5taxuCO9+kgxFcsfEYtQ2t0vf4M9vj4VFX5POnO9Fwc4qlEQsQ9N2ZXuwddZuwDX7i8PfPalSzOxYoAoZTpcC5Ia42DcgzNeoCbn++1/5h+AQqSbAqwtcEG48xSSirFFuyTGVGdmZ2LEndmte423t4tJcJimVNhchxY7bHkX/AGP8ztTCn0lYMrm5uACNBH6Tex+/zK1ajLqA4Ox8FbEffb+YorJNBTTDqmKqXOzi+/stp332t03igfrEfrceAxAHwIpWJNsJ/wARZHVwTdXVvB0EWHngdI/G4xWqVGplgrsSt9jZ9yD8XI+BA6qKf1avjjza8lekbgldIAFgLb82Jt33hiq6Kt0GUKxd0ubhQoHgX4/97zYYj6xVCyJZ9K7Odhrut1GkWIC6t+pH74anTt1tvxGaNJuP/bnpIasqM3HouxnLOrUXN0eqDUbcsF1q5CkHnY3+w6TS0M6Z8SiFSiMCUQ7WQJ7SfJtsOg+ZgabNq37Gx2O9tjvsReWQqviaxu1mKXNl4VbAgAb2t0HMhpt6KjN9lz9X49aw9lb2o/tp2/zSf1g2/KNwPv4mSwlMu4VnVLq27bAFQSAfm1pcfUVBy4drB1RQ6rcohN9AW4FhYcWEpPczLoG5PAv7jwP9z+5msUiJ7djn1fl1aS62cg31JqUgEeCl/O0PzTM1dkKU7JTRKaKxJuiX3a3BJJJt4gL4dkYqwsRyI5+Lj9oSaTS8E2zQ4/MqtX0tXsVEuqBdJJOwbxtsPBPeTZrjlw9MK59XEOARr9yUQeGCflB7bTLvinLXZifJN+I9FR3LVHYXH5rFiTBLey3JpV5Lj6dyBsT7zfRq1O7XszDogP5zzdr2HzCs6xL0qYpUmTQjEbe573ubuAAG8KJW4j6gq+kMOjtoGxfh6i9B/wBo6QfA1kdk9UN6aGxVP1d977X6n/aJp3oalFKkBNTVm+dzcmx7/fmXGMyIp7qrsj6dYREQMV1ADhjdyzoo3v8AtLGvk/qs9aiqJTuNFyU0aVAPtAN99/vKupj6nrh6pGtVCAkGwCm4Pzfe8pSZNJdh+DwaFGZBU0aGB9REB9Wnv+ZTtfjYjixvedfLkXDUhsWdlLhl/wAv1bqGJFrW2tfrbiXOQ5hrRsMyLpVSNrjUp3LEHrJfpqirpW1+4OSrX6rvb4iyNIxT0hmPyypTpK9R0dkIVWRAlw/sUODe41shJ22Hi8zX1RgvRq0URAi1ACWuz6mLMjX1E3ADA8X93M2+PpNiUemrezQy6r29SpYhfsDY/Mo8Rlb42mcTqK1QFFFLjTTNPZ1YdGZw/wAaV8iCZTha0YzFYNU9Smxf10rhFUDWtVSWBI7EEA+dY26hZrk1Whp9VFXWhZSt7EqbMptsG6/cTZZtgFOJw2KIIDPT9ZLWKEaQjntYlQb9hNTmODWqmk2BVg6Na/puv5WHjoR1BIlZkrhuzynD49zh1oKSAHZntcFgbFBcHcAlzaBYyo5shdn62Y6rdueNpbZhlppszAWVmYhb/wCXubL5tx9oFhwATxc8mZrki22t/wCjcGqi+xqYvEKLB2t2O9vud41sfXPLfxC2EhaRFwbvFfgqVrWTAKtV22Yk+I2rcbQ2wG8HxL6iWnRBrpIxa72ColzaFPSIRWZltfSBdS6Bb/oBuAb8kb95Bpsbw50Dpc89+v3lSlTRFEHqlFACCxGzEXJ+N7QmlVIHB6W2/iT0UpgIzEvawIF0K2N7gg7y4x74c0tSamqaroqsxAJPLX2G1j3mcmpaaKUfgoHxRVtlsQbG/I8EcxEqX1DYHuCRuNx/uIDVcljfY7g+Igp2X42sfdfj/eUuNJaEpFp6VX9FUhegL8d+ne8UgDEbe2dkXIqo/BMmEN/ZYrbUA5FxY7i87iypu9jc2t7tlAFrW6/MiqYR1VWe63J/aCYs8b3lJWyGNVze8JXE+2xA556wVE2vHuu1pUoroQQ7gFgp44MJwOYGm+uwuRa9yCAebEcGV6L1nUYXF+L7yKXgdsJxL63Z7mx3NySSfJPMZSpEG4Y36HzDsTWXQSqWFhK9MUOLRpMOhPiWLb88d7+bycUBo1avdf8ALtxBSvU7Qk1QgDryIP4Qn3ZGlPVdG266ibWA8dYIpIGx289YXisf6puUUG3QWhGIqU2pICtnXr/VKutMaVlaHvx238zQZJiqaUWXYMTdjU3U+FHfzKH0GO6o32EJSm9vchsO8GC0HYHO3pOUU3Qtcqetx0kGKran1b87dxO5diEp1AzpqHY9I3MaqPVLoulT0kWPwF4PPHQuzbsyaAf4hOEx+Ip0mWnpAa17/mG1riUtcbA23tt58wrDMdGpvzAbefEGkkCky9GcNR0MjhjYakUWQf8AMssvzUGrqQ2RzrZTwrkAPb5Iv8kzH6xpGobmSJ7uDb4meXyaKckazNc1RnKA3NreD3Bk9HP1YAMwHc95iMTUsQo6depjfxVhvJ3eh+67L3N6grOzD8gO3nzKzE0UsWtYjqOD8wSlmTW0/aTrVuukzJ8TjLRS5b2wWpUHAkdWrYWEJSkAbmQlACSZskkZNt7IEW/MdUp2EaTc7RzHvNN2NOlRDo2l79PYdWR3Zb6SoF9xf3Ei3/1lVe+wmpw+D9OgFJAY3Zh2J6fYWH2g3ao0STAc6rrUKIoAsdyB/YQZMp1+s7lkSmhYKLXY2Nhfgbjf9ohU0NfsYsRnPs0dNr+d7mKLZD72AYDLGeqlNhts7eFIDG/2sPvLTA5Y3vd1ALoCjE2FGzgoy9b7AAbTuX4+9Rn2uwA/t/YSyxIGkajtcX+Ow/2jylZUYxrZksVhbO12BN9yL2J8RTYVcNTc6vaLgbdrAD+0UM2GEfkzdbMC6hW3tK7E7xzmPoWPMqOtmDG0023nDRN7x17GONcQ34GRubSMczp9x2jmXTyJSWiWTmtrASPfCaV1QNK4BFpZfj9SaR+/aDTGitcsd7bRpvwZbU8SAoRl2vzaD56qlxo40iCCgbD0r8cyejlzuwuDzJ8jw1nGvg/zNvSwt7bDbxzBtIqMWxZFh0RNJAv8Sn+p6wptZWBJ/QLXHzLmvl7PsGK/BIg2H+lkVtRuzdSTz8yMldm+DaozaYBXGt7gniV+OwbJydp6LUwKqBsNuBaYn6mpEMN+Te3aNSTIlxYoqEq7i/STNiQFt3kuX5Y1RgDsL3lpmGUKWCp0iajZni0UyAsPiS4d+8bWGg6b8R6LfeRKKoVbHugMHrUpOX0xn4gGRFSQ2iFKO953EPbiO9UCDVXuZcU5StkvRKtQ953EVB0kQS8Y6GVirCxepacareQupj0SaYrsVsJwlUobjvNBRxmsbkzNAG9pbYY6RMeR0tGvE9jcewEqjvDcU2owGohEfEtBN2ybDYoIwPQQ3E5mXIA4lbSp3hIoiXJpEbZc0sWgAilJ6Zik1H5HsZTFxEiHewlnkWGDkqZpf8GVEJtNW0gxb6MalD2kmVrHeX2IcboO9pE2TMFLEbRppCxZX4LFhDciEY7EK4uJXum9pwBhHSES4XDl20zTYDJApBfe/HnxM1QDodYBmkyrPNxq5kyb8FxS8lnm2S/9EsBZgLgdpkcCjO9iCQvM22JzfWhRNyRJcgyYU0LOPc/MhSpbNHBN/SY/EVwz+za3HyJvchr66YJ5HMz2d/TmkmpT+SJe5H/lqeDaxkykn0aQg09luWEYKwiCXkbUZBrVHKrgzFZ5hS9TV/E2bUoJUwI5MpaIkrVGXw7MtrbWEhfGspPexEtsyITYCUNZgYNmLTKrEXZ7mF0XI2nQm8k9Mcysk1TMsQauNoKrWhNUwfRHHoKO3MhfmTLGOZS7JaDKO4nGkdF9o51Mza2OtDWWRE2krGM03MpfcGTYZSd5PUcrJsMlo/E0RIdNlK0iHDIDuY3EUhJKC6dpNUS8L2CKVwQdoVh2PWPqU945EtCUlQqpndJii1RTPZQRk1TTXPzN7VF0PxFFNpdnRBKmYF8P/wBc/M19KkrUreIopLYRS2YPE0gtUjpeW2By5avxFFNG9GaisjRrkq6LWHEyNXLNFW3S8UUlNluKo0mHy3SAwl5hMRcWM7FIkaxSH4gXEHoLp2EUUlDkFCtaN/ECKKMR31xIa+JFoopPktpUZzNqoaZ123iilI5pkTNaMbERRS4mBD6t5IeIopTJQK53iAiilvokKwywl1uIoplLs0XRFonVSKKIkLom0lrVIopLL8Ay1N5NriijYIicyJntFFEMh9SKKKaUI//Z"
print("Predicted: ", get_prediction(p))

1/1 [==============================] - 0s 16ms/step
Predicted:  Positive


In [78]:
o="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEBAQEBMTDxAPDw8PDw8PDw8PEA8PFhIXFxYSFhYZHikhGRsmHhYWIjIiJiosLy8vGCA1OjUuOSkuLy4BCgoKDg0OGRAQGyweHh4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLi4uLv/AABEIALcBFAMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAADAAECBAUGBwj/xAA/EAACAQIEAwUFBgMIAgMAAAABAgADEQQSITEFQVEGExRhcSIygZGhQlJyscHwBxXRFiMzYoKS4fFDwiRTsv/EABoBAAIDAQEAAAAAAAAAAAAAAAABAgMEBQb/xAA0EQACAQIEAwUHBAIDAAAAAAAAAQIDEQQSEyExQVEFYXGRoQYygcHR4fAUIkKxI1IVM4L/2gAMAwEAAhEDEQA/ADCSEyl4gJL+YievyM8jqRNQGTBEyhjxHGPENNj1YmsDHuJljiA84/8AMBFkkPVj1NUMIgRMocQWOOILFpyHqw6mtcRXEy/5gvWN/MFi05D1YdTVuIriZnj16yXj16w05Bqx6mjcRXEyzj16x/Hr1jySDVj1NO4iuJlrxBesRx69YZJBqx6mpcSJtM3x69YvHL1hkkLVj1NEgSJQTP8AHL1j+OXrHkYakepcZBEKYlLxq9YvHL1hlYZ4dS01MSLU5VOOXrG8aOseVkdSHUK1GOKVpKixOo1+UetWA30hd3sSSVrgyTI5oJsUvWIYpeseUSqrqEFOEFKV/GL+xJeNWNqQakepM04N6UmKl9lJ+Bjsr/cPygiLVxUUjtRkVzfcb/aZGpicvvAj1ELO+wX6hO5ilfx4iiyyDPE48YlpJcS0N3Mj3MheRpvDoP4to64wyPcxu5jvIVodAhxhjHGGQ7mI0YXkGWBPxjRxjGg1owhox3YmodCXjDF40wfcxdzC8hZYBPGmN40wXcxdzC8gy0+gXxrRDGNBd1HFOGaQ8sOgXxbSYxZgMklkks0iOWHQJ4sxxizA5I2SGZhkh0LHizF4owOSNlhmYskegbxRi8UYErGtFnY8kSx4oxLijARos7DJHoauH4iQN7QeKxpPO8zooszI6SD+IMcYgwMQjzSHlXQMcQYkxBvAxR3YZV0NnB48r/zNZOMLzt85yEQEhKObiEVKPBnZ0uLL5fOZfFsaG2t85hSJiUbO6CTnJWbLHfmKV7RSy7FlRbyRd3DWitMyqlOYGKcfuoW0QktQjmYEUo/cw9orQ1AzsAKMfuoe0eGqJzYDuYu5EPFDVFmYAUBF3EPFDUHmYHuIu4ENGjdUWZgu4EXcQ0UWsgzsD3AkfDiWZGQ10GdgO4i8PLEUesPMyscPIGhLkiRHrIedlPuIu4lu0fLDXQ9RlTuI3cy5ljZYa4ajKncx+5lrLHyxOuhajKncxGjLWWLLFrj1GVO6j91LOWLLJaoajKppRu7lzLIlI9QeoVe7ilnJFFqMecnljgSVo4EzblFyOWILJ2ii3C41o9oo8MrELLFliigoMQ1orR7RWksoxrRWkssWWOwrkbSzgMA9ZxTpi7G51IUADckmBtN7scP/AJH+g/mJRiqro0Z1FvlTZowlJVq0Kb2TYm7F4vkiN6VKX6kQFTspjBvQY/hKP/8AkmeioTH70jmZ5Ze0tVcYJ+Z6Z+z9B8JSXl9Dy2rwbEL71GqPVKn9JUqUWX3ly/iB/WevDFOPtGOce3Ox9QDLY+00edP1+xTL2eX8aj+KT+aPHLRWnrj10Pv0qT/ipqf0lOrSwx3w1L/SuX8pevaPD84v88Uih+z1XlNeX3Z5faK09Eq8LwR3w+X8NSoP1lWpwLAnYVU9GB/MGXx9ocE+N18F9SmXYOKXDK/i/ocLaK07Kr2awp92s6fiQN+REqv2VX7OJpn8SMv6maodr4GXCp5qX0t6lE+x8ZH+F/Br6o5i0Vp0D9l6g92rRf0qWP1tKzcAxA/8eb8Lo35GaYYvDVPdqRf/AKX1MtTBYmHvU5eV/wCjKtI2mjU4PXG9J/8AaSPpIfy2rzQj6TSlF7poodOa4pr4Mo2itL1PhlU/ZPxhhwat92DUVzDTm+CfkZuWNlmm3CKo+zHXgtU/ZheK5hp1OGV+Rl5Yss2k4BVPKGTs1UPl8I88OpJUKr/iznssU6P+y9Tr9Io88OpL9PW/1MHuvKS7qemf2bXp9Ij2aXp9Jg10bX2RU/2PM+6jigTsD8p6SOzK9PpLdLs2oG30jdewl2TU5y9Dy1cM3IfSH/ltS18h+c9OXgCg7CXRwdbbCL9SWx7IfOXoeQNg3G6n5SPhm6H5T1ipwVeglZ+BL0Ef6ki+yHykea0uHu2ymaWH4CSNQZ6Dh+CqOUtDho6RPEFtPsuMfe3POD2cJ2vG/s1U/YnpaYESfhVkf1BZ/wAXS6HmP9mnmpwHhDUqmY81y/M/8TuPDLGrYQAXHWZ8XVc6E49Uy/Ddn06VaM0t0ygwsIMwtYwRngsQ/wBzsemitiBg2hDINM9yVgTys8svAVIpsaRVqSs5lmpKtSUZnxJWAVDAuYV4F5bTYWK1VoDvD1PzhqkARLr3FYMmIf77D0Yy1RxtXbOx9Tf85TpiWcMvtCbaLtvwISSezOw7O4POjM2vtAC9umv5zbXhq9JR4K2Sio63Y/Ga9KuJ7GlKapxu+RxqkY53ZFVuFr0jDhq9JfasIF8QBLFObIacegJOHL0hRgV6QfjhC08YIm5jyxG8EOkUn4sdYorzDLEJcSYtMDx5hF4iZW3HqXZGbYAkriYf8xMc8QMV49QyM12Ig3rCZRxxgnxJMeaI1TZpvXEitYTLNYxhWMFU7h6ZseIETYoTGNQ9ZFqh6wc+4NM0mxgkXxczLxiZHVZLTReGM1lrxN1I8pjwtFtfhKq9WSpSv0Y401mQVjImSkTPE1HdnSRAwbQjGcD2t7f+HrmhQRajUzaqzkhQ2+QW3PnLMPQnXllpq7FOairs7V4CrKfZzjS4vDrXUZSSVdL3yVRuL8+R9CJcrSmpBwvGXFDi090VKkqvLFSAeY73ZYAaV6ssGVMRiEBCllDNsCQCfSXwEwNSQcbel/rJvJVyPZtyAB9d/wBZdHgIhTEmKmVgd8oLH8hGpwjL7NVvwKPmCZsjG8bePor/ACIs3l7S0VAALMQALBbD62m1wzGrWQOpIF7EHQqehnn+Hp7ALcnYWJJnQ4HhrhbO5RTa6KbfMzt/ratfahHL38fNtJeSMego7zZ0dXHU10aoB8RAeOpt7tRSfWUF4dSH2AfNrsfrK2O4QhF6YyMPunT5R5cbFXU7934gy0uFjWLxd7Oc4ZxFw3dPqRses3Ea4uOcuwuMdZNPaS4kalHKyx38UDaKa88iGRB1pSXcyFNpPNK9QnkIMLSaLeDcQ9Ix6oZBdzJCjCh5IPDUuJwBdxG7iHzyQaS1COUqmjI9zC1yeUClQyLqksg4pRd1CM0bUxOq0PJch3cfu7axlU3lip7vxlOIqt0pruY4wSkgIjNHvGaeQqM3IBWNgT0BM+b69Yu7VD7zszsf8zG5/OfSZE8g4p/DvFeJZaKq1B3YpWNRFVEJuAy3zXF7aA3tOt2PWpwzqTSvbyVzNiIt2sbf8JlbwuJYiytivZ0sCRSTMR9B8J2VQwPBuFJhsPTw6G4pg3Yixdybs5HmTLDzlYyqqlaUlwbNFKNopFOoIB5Zqyu0wcy0zOM40UKNSqdcik26nkPnaeOY3FPVqNUqEs7G5PTyHQCem9vUJwVbLyNJj+EVFJ+k8sp0mZlRAWZyFVRuWJsBPV9iU4qnKfO9vgkn8zDinvY9S4FXaphaDsbs1Fcx5kjS/wBJdMHgsN3VKlSGvdUkS/Ugan53hLTi1GpTbXC7t4XNcU7K4akNRLtHDM9MKgJapUPwAvqeg2lWiNz0nXdnqYWkL7m35f1M6WDpak3F7Kz9X9iupLKrg+G8IWkPvOd2/QeUu9xLd4xM9BBxpxUYqyRiknJ3bK3cR+5hXeVKuKyhidgCYnUGoHPVMNfEsALZM1tOV5rcOH+IOjD6qIHg+FOVqrXvVYvY7AXJ/fwkcLW1cj7Tk/AC36GcehK2Nk1w3NEt4I1MkUam+kU7OoZ8hIJCrSjAgbSD4vlKronuF7qSFO0rh+cg9ZoZkGVlh2EVOVaTEmWlkUydg0SyvVewldKxJjuLKaEY0xB0pGtVtC4ib6RkqiRWqCIKqBBMC8rAxq1rfETMQEHeWVqXsPjIV5f4peDCMd0cz/EnGVKWCzUmZC1emhZCVYLZm0I21UTW7O4xq+Dw1Z/eqUKbN5tbU/O8j2swC18FiabC57h6qeVRFLIfmB8CYuytRGwOENMWTwtEAb2sgBHzBnm6zi8OlbdSfqvt6Ghe/wDA0TIsZNoJphuWAakr1DLDyvVkGySRWqys8sPA1JXHjckUsTTDAqwDKwIZTqCDuDMTAdnsNQc1KaHPqFZ2Z8gO+W+3rvN2pKrzXSrTjFxi2k+PeQcE3dgnjKNYHiOLWlTeq2yKTbmTyA9TYfGYHZHH1q9Ws9RiVyrZfsISTYKPQGaaOHlKEqnKPr3IUppSUep19M2Fubaf7tD9LzsMLTsq+gnKYanmq0V6sT8hOzyzsYCPvPwXzKavIkHjM+kgVjhZ0LspsBFbWVa1Pv3yL/hIb1G+833B+sp8X4mATTpm7bMw2HkPOa6VUp0VYCy5RlUbknkOpmbFV3Rjt7z9AiszB49rKKae+/sIOg5n0AkqeEVQFHIASWEw7XNWp/iMLAf/AFp0Hn1lrLI4PDOnG8uL/PPqOU+hXyRQ2WKbbELlOo9jaECC17QVKqGNiLHrylhbESNiVwLPbaFVgRJ9yI6URCzC6BLYawYqljpLTUBePTw9orMd0AyX3iSlDlZLJYaSVhXAqpleqL+ctimY9OhbziyhmsZ9jsIXuTaTbE2a2WWEIaFguUcpEs002Mt90IqiWEhWj/jl4MIy3RWdQQQdiLH0M4n+GGKtRxGCY+3gsTUQA792WP8A7h/mJ3BnlPFcZ/L+OtWbShiQr1OndVQA7f6aiFvT1nnqEHVhUpLjZSXjH6pv42L5vK0z1AyDSZNxcagwbTmFwB5XqSw8q1DK5kkAYyvUhmgKkIPYZVqmVmlmpKeJrKis7kKqgszHYAc5fBAYXbFr0FoDWpiK1JEHmHDEny0+s0eCcNWggpprrdmO7N1mDwJmxWKfFsCKdG9Kgp5Ejf1sbnzYdJ1tAazp1M1JRoX75eL5fBepTG0m5+Xga/Z+jmxQ6JTLfG//ADOvyTmuyqf3mIf8CD4Ak/pN2viVRS7kKo3J/es7ODWWl4t/3b5GervIK9gCSbAaknQATleMcdL3p0dE2L7FvToJS4zxtq5yrdaQOi8282/pBYHBs5AAzMdl8uvkJLEYpUnkjvPp08foRhDNvyFh6JNtCbmwA3Y9BOt4fgSoVqurKLIv2aY/U+clwrh609feqEat08lHITQtIUMPvqVN5f1+d2y5b7jnLkiEYybQQN5sKhZooi4ihYDOo6EWF77fGWlokaMQCekqU3K8rgWGbbSWUrh2IH2SN77yu/ItUSzRTLf63kxb05yjxXHmlTLZS99dOUofzVXNrG6gAkqR7XT6iGdIFFs6AkaHfeUGxrBtQMttTtaHoVvZAeyk2Fhrr6zN47hc6XU3IIsAd+og27CSV9y3huII5Njtz5GWBi0zBQdTtrMXhmBZHCtYpa/XXpD4/ADOrqchQgnoR0tBSY3FcjavraQetZ1XrMyjjybs2gBspOmnWFavnZWGjLz5GSjJMjKLRqd0OgkSgGwj0nvE9QCTID2kK2i/EfnGGIWKtUBU/D85VX/6peD/AKJR4ormcL/Fjgxq4ZMSgu+FJz9Th3tmPwIU+mad20HVQMCrAMrAqynUMpFiDPKUqzo1VNcjXKOaNjzf+Gva4Mq4LENZlAXD1GPvqNqR/wAw5dR6a+huvSeG9sOzL4KtYXag5JoVfLfIx5OPrv6bPZz+IVWiFp4kGsgFhUH+KB5/e9d/WdHGdn6v+bD7p72+a+a68EV061v2zPUaglV5SwHa7CVx7FVb/dY5XHqp1lt8TTOoInn6sXF2krPvTRrW/AA8rVITE42kouzAAcyQBOW4x23w1O4pnvm6UtRfzfb5XllChUqu0ItilJR952NrFVFRS7kKqgkliAAOpM8645xZ8bWXDYe/dlvTOR/5G6KN7fraUcfxPE46oKYBIJulFPdFvtMedup0HlO07O8CXDJrZqzgd442H+RfL8/lO3GhHAxz1LSqPguS73+ceHVZnJ1XljtHmy7w3ArRpJSTZBa/NjuWPmTcy/hxrAmJ6lvZ67/mZgpzbqZpbs02SVkbPDuL0qFBmY5qjvUIpqRm3yi/3RpzmJj+K1K7Xbb7KC+VfTqfOUqeCLEtsL31mlgcPdglMXY/aPIcz5CdZ42cacYU1ZtJX4t7cltbx9TLpqUm2F4bgGZgoF3PX3UHUzueG8PSkll1J1dzux/p5Spw7ArTQKu51ZubH98pqqLC02YPDKmsz95/H873z4bIhVnfZDZBfSSjyJm8oFaRFo5jWgAGphwTeKFuYoAUcZhhoM1rDS24JvygeE4YoHzMXY1CxYrk02A84bEVA2cgEEj3wR7oPI8pTGJ1NNPaKKGcBidG13+kytq5oSdg+NQVLizEKdgTYtyvA4NLDMaZvew56ev72hO9qMNEI92wO2+94VqYLXY30ygKfZ0POHEOBJ0zZWIyjkL21vAV3vZVGwHtb2N9oZU3tdjyAAsvnHvqdNSNwNDJoTBooFs1rnUgG5tDViCL6Dnqd7SGGq5gxsBZ8qjnoNyZaoUidWAubb+fnyi7wMc0DUUgoWAOn9JZ4dgXKgNemFPsggX08pqkWAA5Hfl57CFKG2mvn8NNIZeZG+1gNLS/tZtNNJF61h1vtpeFqA5b21vvl6CRWnZTzOW5ufL6STkxJICKwI0X6X5R+8Y6FQB5fDeRwqFr7WI2O1rdectOltxe25vYD4SqeaUWu4mkkysRBtD92b9Lbk7Dz9IEm4uOevTSeZr0J03+5WNEWnwKPFOH08RSajWXPTfcbEEbEEagjqJ5t2h/hs63bBt3i6k0apCuByCts3xtPU3gHjoYyrQ2i9unIU6alxPnriHCa1G4rUKtO3N6bBfg2x+Bmea5XQMQOgYifR7tylKrRT7if7F/pOlHttW3h5P7FLw3eeA4fC1ax/u6dSsf8iPU/IGdHwvsLXexrkYZOa6PVI8lBsvxPwnqdRraDTyGglWqZRV7bqSVqcVHx3+xOOGXFsyuHcLo4dMlFMt/eY+07/ibn6bSwRCNIETmynKTzSd2zVGKWyGMoV6mvqZdqTE4jiwhuxAAHOW0YuTshSdkXKmJCKSxsoEN2e7WUaZIqU2Gb7a+0176AjpOJ4hxM1TYeygvYdfMwFN7fQa31ndwuF0/3z3l/SMs6l9ke88P4zRrKDRcE/dOjj1BmjTczwvA4rKVJNgDoRcENa9t9Z0eB4/WAGWtU0GgJDAXOu41nRi0UtM7DiPawUqrUlp94VbLmz5RfncW25TS4PxpcQSoBVwL5Trcaag7bzzIZmZ6mfMzsXYtoCb+Uv8ACOKVKFQVARa/trmBBHMC3/cavcVlY9RZDfWQKTNwfHKdVQ6lmFwLWNwSL2PnNDvgRoRrHeLexGz5j3jRhUWKMCo7hVAb3SFFtwQSBtD4Q72GVbW0AvawyxRTO9mXcUM9wCNbC1tdSTvrK2dBc2vtmuTa4PSNFCTsOKuWFux0OWwBYDpCZLXA1bUa9Tyv8YopKyIthqFDa4GvtcuUJVbKLbXNrWGvlHij4RbIriDCE6nRSb2uTaw/6+ssMbC/mPrHijiuJFsCtS+oFxY25eREmDyIJOttdCANb6+cUUimTewQNYnQcr8jz5f8zO4w7hP7tgCCLggnMNCRe+nOKKKT2Y4rcz8LgaxGdma/skUzUuoLC4vr7R156CWAtXMM41ZLscwPtA2/f7u8Uz4mlGdJxZOMncTeu2+8C5iinmK9NQexpRWcwNUxoplW1yRUqtvK7x4o0MrMIrRRS0mVsUbCcT2oc5kO4Ie4+WvrvHinY7KinUfcjNiPdMVX/wC7fpLVL52HprtHineZkLe4W4zJcaE2ym+nrrebFFQMyj7QAFybb7eXr5xRSDJIt0KNlCiwtvYC1+fxud/Mzf4Dw8uy1DbIQGTS5YDa/T/gRRRpCOrN7gX9mw0sLlr669LSdrXPPmdNP6xRSXFsXIgFH7EUUUaYH//Z"
print("Predicted: ", get_prediction(o))

1/1 [==============================] - 0s 17ms/step
Predicted:  Positive


In [79]:
s="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYSFRgVFRYVGBgREhEYGBIYGBgRGBIZGBQZGRgYGBocIS4lHB4rIRgYJjgmKy8xNjU1HCQ7QDs0Py40NTEBDAwMEA8QHxISHzQkJCE0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIALcBEwMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAAAQIDBAUGBwj/xABGEAACAQMCAwUFAgsGBAcAAAABAgADBBESIQUxQQYHE1FhIjJxgZEUoUJSYnKCorGywdHwFSUzc5LhFlOD0zVEVJOztML/xAAaAQADAQEBAQAAAAAAAAAAAAAAAQIDBAUG/8QAIREAAgICAwEBAAMAAAAAAAAAAAECERIhAzFRQRMUMmH/2gAMAwEAAhEDEQA/ALltRT+tpbWgpkgtcbSVLcg5zk+U8jJPdnuY0QGzXGCAf4ShWsynqJtqDnB+kLhBjB+vlKhNpmcoJmJQojO8m1tnbIlxKQ1Z6ftkj0hjbM0y2RhS0ZzITzh4Rxy2lwUovh9JWROJVVJZt6KYOr6S/bWY05I3PLbOJF9m3PpFlY1Gim9ID3c/SOVPjLv2aKEhkGJSKYjGJPOaS0CdgM5ja9mVO8E0JxM3TADEteFDw5dk4lNx6Rm8umnImTEExOJGrYwTIXeTOCZCyRollSqSZXalLxZVPtcvLqYlzcpo9kDeaJv4Q0vpSr2pQAtyPLrKjpLgqlwAfwYrUgBky02uzNpPozikTEvKgblB6ErIjEoiJLTUZG1GPJCcCGJJTRMjYER2iHFjSIhjtJ8oYjJoiaQsJZKyN0gS0U9EJLiECaPUQsgfOdpKDmSKR5TxIxo+kbK4ptj+PWNK+c0FAMZUtNUpOuyG0Z7ECSg5/lLA4eIjWWOUvJC0QijmPShLATSN4Iw6xqViokU7YiqMfOPRMyZaMCHJIrqgxyiNQl0UY4Uo6JzRVt8LvjeR1FLHJ6y61OR6I6BSXZQehInozSZJEySkyrTM16Ua9DbfaaDUpBcYxvC2KkZjoozMm9vNOQAP5S9eVMZAmRWpFt5pD1mM9aRSqVdWSTIfEJ2ztnlLL2pPIRycOYzoUoo5nGTJbOoAPWWdGsZIlux4YVHtbiXWttPTYTOUlejWMHWzHp0cbR5pSW6rBRgSgt+QcYzBXITqJM6YlWq4xtEva5ImeMzSMfTOU66J/FMQtmRlvSS0kbmB9ZdJGdtiODzkBlo2ud2OTEe3AhlQ8bKuqJziumJFGnZEojtEIsIyKPTFSOVIBZIqzx0e+wVJKojkSTJTlJGMpEarHhRLCUZMLeVhZjLkRSaiDBLQS94MVaca4yf1daZClHElWnLC05JoErCjGXIV0pxrpJ6jYkRMdCTfZCUjSkcz4jfEhRqrIykYUkxcesbsY6LUmV2p5kNS3zL4SIacWJSmYtfhgaQLwQdTNw0zGlTBKgcr+GQOEIIn9nYORNRpExjp+hkvCqaeOkpXQZpqlpGwHURNME4/Tj76kxMq0bTedhXs1brIVsgs0jyNaM5cabs56pYloqcIHUzdqIPKRtTUjbIx98vN0RgrMprHB2wAJHUpYl+ojHko+JlY0Hzk7wUgcSk6Y/nK7vmadS3dhg8vKVntQkuMk+yJRfwoVKQxmVHWaTrkSCqgxy3l9EdmfqhJtPxhHkTiepKkkVZoLY+slSyA5zy4wkz0HzxKCJLNMecui2HlF8H5TRQaMJcyZEkkzHeFDwxNEmZOSY3UIquI4Ux5SQKJdMTkhAwikiKRGNE1IhEbSIsBJtAiaB5QxZomiuzk8gIiseqiWdIgYKLKyXhHiNKiSExuRHiCYzEaRHswgBmKikyErGMksERhMKZVlV0zGNTls4jSIgsomnI2py8ySNkgMptTkTJLjiV2WNRE5FN6PnInwJbYGQvTlKPosin4mIgqqdiOclanGNRlYIjJobXpyk9EHmJeZD5yJqcFGgc0ZdS1UbynWpzbanKlemPnLWibTMjPpCWzRhHURWz13VHK0qh49XnEpFOBZiyJXkgaaKSM2mgMSLmAlWgFWKYCBlpiGkxDFhAY3EIsIxjSImmOhAdjCsYVkpiEQAhKRNMlxExALItMbiT4jSIUPJkJHpGkyciNKwxQ8mV2zImEXiF4lBGq1XVEQZZ2OAN8AepJ2AG5nGt3m2OvTi40/wDM8MafjjVr/Vgo30gcku2dcUjCkdZXNOui1KTq6OMq6nIPQ/Ag7EHcTmOP9ubeyrtQqU67MgQlkVCuHUMMFnB6+UaTYnJHQskiZBOPPeha/wDJuf8ATS/7k0OBds6F9V8GmlZW0M+XVAuFIB91yc+0OkeLJyRtPSEhen5STiN7Tt0L1XVEBA1McbnkAOZPoN5z9LttYu2kVtOeTOjovzZlwPniLFvoMq7Nd1PSVqimXjvuORGQeefhMLtB2gpWYGvUzuMrTTBYjONRzso9T8sxpMVpFzQZC9Oc1b9vqTNh6TopPvgipp9SAAcfDM6pXDqGUhldQysNwwIyCD5RuLXYsvCkyQljwYskrI7ZAZOiy2KIHSO8MeU4I8EvTSXKmQoskAjwojgJsuNmbkMCxwEWE0UKJsSITFhLoCrfXaUEapUYKiDLM3ID+J6Y6zgOId7NBGIpUHqAH3mYUgfUDBP1xKPfJxU6qVsrbaTUcDqSSqZ+ADH5zzzhfA7i71fZ6TVAmNRXGF8hknn6c5Sjq2Juj2bsv3g2964pFWo1G91XIZah/FVxj2vQgZ6ZnYz5YbXTf8JHRvVWRlP1BBHyxPo7sjxf7ZaUq594ppf89Dpb6kZ+YjcaBSs2YkdDEkobiGI6eB9qe0t9RvalN7mqVt7htKBvDVlDakDBMAgrjnGlYm6PdqtRUUsxCqgLMxOAoAyST0AEwbbtjZVayW9Our1KhYKEV2XIUsfbxp5Kes3EK1kB5pVpg/FXX+RnzbYubK+QscG1u1DH0Sppf7gY1GxSbXR9F8RvVt6T1qhwlJGdjzOFGcAdSeQHmZ4ZxrvEva7kpUNBM+zTp4GB01PjUzefIegnpPe3XKcPYA/4laih9Rkv/wDgTyfsX2ZPEq5payiJTLvUA1EDIAVR+MSfuJ6YlxSq2KTd0dZ2I7xavirQvW1pUYKlcgK1NicKHIADITtk7jmTjl62Z4B2u7GVLGulJNVZLjPhMqHU5HvIVGfbGRy5g58wPa+zK1xa0VuRistNVcag59nZSxG2oqATjO+YpJdoqLfTPK+93jRq3AtlPsWqqWHRqrrnJ89KMoH5zTg/AfTr0PoB0mppbQG/F1Yxn0mj2mrNVvbluZe6rhflUZUH0AE99pcGpJbC00jwhR8Mr+MNOGY/lE5bPnvLcsUjNLJs8w7ouMFKz2rH2KyF0B/BdB7WPzkyT+YJk96H/iFT/Lof/GJ13Y/u6e1qpc16w10iStOmMg5Vl9t28wx2A+c5PvSH94P/AJVD9wQVN6G7x2ZFt2XvKqK6W1RkqKGVxpwwPIjedX3e8Aube711qDongVF1NjGSyEDY+hmt2b7c2VC1oUqjuHpUkVgKbsAQOQIG86Tgnaa2vWdKDsxRQzBkangE4HMbyZN+Dil6eXd5XE2q3bU8nRahUUdNRUM7fHcL+jMLiPBbi2VHrUnRanuMdJB2zg4J0nG+GwefkZa7ZIVv7kdfHcjr72GXb4ETt6naG04vRFtWc21Rnpt7QBXWp/Ac7b5I9rB35GV0kT22J3acQNSg9Fjk2zrp9EqAlV+TK/wGBOH7QV3ur2pjcvXNJAT0VvDQemcA/Mz1Ts52Up2Bdkeo5qKgbXowNJJGAqjzPWeScWR6N1V5q9K6qMp6grULI37DEqbdDldKy12i7N1bLQXZXWpkB0zgMNypB9NweuDynXd392XtmQn/AAajBevssAw+8vKFLtVbXqpSv6ekK6sKiM4TUFK5YKQyD2j1I88TtOG8KoUFPgIqq+kkqS4fb2Tkk52P3xSeqYJeDsQk/hwkFUd/CJFmSYwixISxBCEIgMrtFxYWdu9crq0BcJnGoswUb9BvI+y/GxfW61wApZnVkB1aGU4xn4YPzmB3qcW8C08IBSbpim++lVAJIHnnTv0zPO7Pta9O0SytqboajHxaiku9QtgMKa/g5AA/lGk2VqhO865WtetUTdCoQP0dqfsuV9Ads9ceU6Lun7SW9ClUt6zpTdqpdXchFcFVXTqOwI09fOVe0fZivUsFuGpeEbQBUtQdbJQyS7OebOWOo8ts7DlPNSs0ilKNEN0z23j/AHfUuIXJuRXCpUVdQpgOXYbFg2cDIC9DvOq7OcBpWFLwaJcqXLku2oliACdgAPdGwE+dOG8VrWra6NR0YHmrFQfiOTD0M9r7ve2f9oKadXStekMnGwqpnGsDoQcZHLcEeQmUWhppnbQhCSUJPCe9+x8O/wBYG1xSpvn8pcof3B9Z7vPL++yxzSt64H+HUqUyfR1DL+4frKi6ZMujqe72+8fh9u3VKZpn/psUH3KD854/3m2PhcRrbYFUpUX11oNR/wBQed53KXuu3rUSd6VZXA8ldcftQ/WZffdY4e3rge/Temx/MYMv77fSUtSE9ov9sK5uuBUq3MhbRmP5QPhv+sTMTuVrgXNdOr26sP0KgB/fE1+w6/bOC3FtzZPHRB6lRVT9cn6TiO7niIt+IUGJwtRjSb/qDSv6+iNdNCb2mfQLICQcDIzg43GeeD0i6Yy9qFKbuMZSm7AHkSqkjPptOJ7J95FO9qpQei9OpU1BSGFRCVUtudmXZT0PxmaTZo5JOjyGttetnpeNn/3959Juu8+ce1lu1G+uV6rdVWX4M5dPuZZ9BUeLU3thdah4Ro+KX6BdOps+o3GPMYmk90RB9llhPDe9Uf3g/wDlUP3Jd7Ddsbs3SUC3iU7msc03yxpByWY025jAz7O425DnKnewP7wb/JofumEVjIJStGFbdmruoiultWdKgyrqhKsPMGd33XcFuLetWavRqUw9JApdSoYh8kCdX2Ju6YsLYF0BFFQQXUEbnmCZupcox0q6EnoGVj9AYSk9ocUtM817y+yr1H+10ELnQFrU1GW9kYVwBu22AfzV9ceX856vdd5mmuaP2YKErmm7tVzpC1NDNpCdME85c7xezVCpbVLlUVK1EazUUafEAYBg4GzHHInfIG+MxptUmS1e0Z/djxtqqPbOxZqCqyMdyUJwVPnpOMejAdIdu+xz3DfaLcAuVAelsviaRhXQnbVgAEHmAOo353uuz9uOOttWH61P+OJPw7t7dPcUlqsi0zWRaiqgXClgrZLZIxnPPpDF5WgtVs4uvRemxR1ZGXmjAow+IO4npPdjxE1KVSgxJ8AqyZ6I+cqPQMpP6U0e8+zRrTxGA10nphH/AAsM4Vkz1GCTj8kGc53VKfGrnoKKZ+Jfb9hg3asXTPR/DhJdMJkVbOtEWLCZRiUEIQlCCEI1jgZPIbxgeU99FUFrZB7wWqxHUAlQD9VP0mp3P8PKWz1WUZq1ToJAzpVQCQeeCSR8p572j4g99dMxyS7hEQe0QucIigc/4knznuvALD7NbUaPWnSRT+djLfeTH8KaovsoIwdwdiDvmebdpO61KzNUtXFIsSTScE08/kkboPTB+U7jtFxBra2qV0QOaSatBOkEAjJzg8hk/KeUWHedcvdUjVZEoeIA6KuBpPsklmyds559I1fwRzXH+yF3YjVVp5TIHiqQ678skbr88St2R4obW8oVQcBaiq/qjnSwPyJn0ZxCglWi6Pg03puGzuCpU5P8cz5cJwdjyO31lptppkNU0fV0JBaklFJ5lEJ+OkZk4ExTbLCct3kWHj8PrgDJpqtQfoMGb9XVOpkdWmGUqRlXUqR5gjBH0lWB4d3P8SFK9NNjgXNNkHkXUh1+4OPnOz75qamxRjzW6p6fXKOCPpv8p5L2g4W/D7p6WWU0n1JUGQSudSOpHXGNxyIPlE412kubwItxVZxT90EKgB8yFAyfU7zXG2miLpUd/wBx9c6rpOhWg+PUFx/ETj+3nBmsb6oqgqrt4tJht7LsSAD+S2V/RE7TuQtGAuaxHsk0qanzI1M30BT6zoe8a0s7qlorXFCjWpamps7qGUkbqy+9pbA5DmAd8YKupBVxOPue9Rqlm1FqTfaHpNTasGAT2l0lwOYbBzjlnr0nPd2dIvxK3x+CarH0Aov/ALfWcvVTSxXIOliNQ3DYOMg9RLHDuI1bZi9F2puVK60OG0kgkA9OQ5S8aTr6K72d93x8FKV0ulHsXChHPlUQbZ+KAf6DPPReVPD8PW+gnPha20E5znRnTn1xFu72pWbVVqO7fjO7VD9WJkEcVSoT2zV7McZ+xXKXHhip4evCFtG7KVLA4O4BPTrLfbbjaX1z46KyhqNJSjYyrLkMMjYjlvOfhHW7AbpHkJ1Hdqo/tK35f+Y/+vVnMyS2uHpsHR2R1ziojFGXIKnDLuMgkbecHtUNHQd4XDTb39YEezXbxU8mFTdv19YjeIds7q4txbOyFMKGcKRUqBSCodtWDuBnAGcb9c5PEOK17nT49V6mgEIXOsqDjOCd98D6SnEl6I9F7pOGkvWuSNlQUkPmzEO+PgFT/VOZ7b8JNrd1FI9iq7VEPQq5yQPzWJXHoPOVF7RXK01opWdKdP3Up4o45kksgDMSSSck5Jkd5xy4rIKdWq1REOVDhHZT5hyNefnCndj1VEd5xWvWREq1XdKfuKxyF2xn1ONsnM9K7s+FmlbvWcYa5ZdPnoQHST8SzH4YM8plqw4jVtzqo1HQnnoYgN+cvJvmISVqgR79CeH/APFV7/6mp+r/AChM8GVkfTsJBXuQnPPylZOK0ycE6fU4APznO+WCeLezRQk1aRoQjdQiFxBzSJofOW7fccFnatpOKlcFKfmMj2m+QPPzInS+IJwPDrP+0717qrvQtX0UUPusVOQSPL8I/FRyEM0Uovs4ThPB6lrf2IqjDV6lvVC9VU1SAGHQ+znHTInvs8zs7und8Re7r1ESjZexR1OE1nJwwyd+rfNROlu+3VjT51gxHRFZ/vxj748kwcZeHQ3FBaiMjDK1FZWHmrAgj6GfNHH+EvaXFSi2xpuQG5a15qw9CMGetXnerbr/AIdGs/5xSmD9Cx+6ef8AbTtSOJFGNutNk2Dh2d2Xf2G2AIyc+m/mZpBuyHHRm/8AFl54H2b7Q/hadOjbOn8TXjVp6YzjG3KVez3Dmu7mlRXJNSooPoucu3yUE/KUSkdRqvTOpGdDgjUhKHB5jI3xNaVaI3ez6kq3CUx7TIoH4zBQPrMm67W2NP3rq326LUVz9FzPm2o7McsST5kkn743Ej8/9HkfQZ7xuGg4+0fPw6mPrpmbxLvWsqRxTWrW/KVfDUemXwfunhuImJX5oWR2fbbtrS4kij7LodPcr+JqYDO6soTDKfjseXXPFRcQxLikuhXZ0lp24u6FBbagUpIoPtouHYn3mLMT7R8xj0xOcdyxLMSSxJLE5LEnJJJ5n1jcQjSS6AIQhAAhCEACEIQAIQhAAhCEACEIQAIQhAAhCEAPqO4AzMu4UeU0K1TPOULgZ5T5rmactHq8KoucNugVxy04GPTpM7iPaIU20qA2DuSdvgMTLux8RMO5SXDkbSRr/Gjbk9m/xLtYrIyqrAspXUT7uRgkACce92VXSruE39gOwXfnsNpFcA+szazGdUY39E1GCpIfVCf0TKrhP6MgrORKj1Z0wg/TCfIvC0+j0+sgfT5D7pVerI2ebKJzy5F4Tvp8v2SJtPlIi8aWmiiZOQ848o048owtGkxpEOQ44iHEbmITKoVikiG0SJGIUxIkIxBCJEgMWJCEACEIQAIQhAAhCEACEIQAIQhAAhCEAPo5bsMMdZTr1ys5ylxHB5zSW8FRfUT5mUGuz34xinonqXKtzmZdUc8jK9y5U7SuL2VGDW0aWloq3SkcxMusAZtPcBpn3NEHlOvjddmU430Y9VP6Eo1k+HzGJoXFMj+sylUedkGcPIvSk6en03kDS039YkLn5/GdEWckkQGMMkYen0jCJaM2hMxMxSI2MkXMTMIkYC5iZiQhQ6DMIQgAQhCABCEIAEIQgAQhCABCEIAEIQgAQhCABCEIAd09zvnz3lm3vipyOkITypRVHsRk7NR6wdc/1mZFztCEy41s2l/Upm5PIxTc/MQhOjFGGTB6gb+t/wDeUq9MHeEI46ZMtozq1LErMIkJ1ROGfYwyNhCE0RkxpiQhKJEhCEYhIkIRjQQhCABCEIgCEIQAIQhAAhCEACEIQAIQhAAhCEACEIQA/9k="
print("Predicted: ", get_prediction(s))

1/1 [==============================] - 0s 28ms/step
Predicted:  Positive
